In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("../cache/")


# \n을 기준으로 문장을 잘라버림.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vector_store = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="refine",
    retriever=vector_store.as_retriever()
)

In [10]:
chain.run("윈스턴 씨는 어디 사십니까?")

"윈스턴 씨는 조지 오웰의 소설 '1984'에서 주인공으로, Victory Mansions라는 아파트에서 살고 있습니다. 이 아파트는 런던에 위치해 있으며, 이야기 속에서는 빅 브라더(Big Brother)라는 국가의 지도자가 시민들을 감시하고 있습니다. 윈스턴은 진실부에서 일하며, 텔레스크린이라는 기계적인 음성을 듣는 일을 하고 있습니다. 텔레스크린은 시민들을 계속 감시하고 있어서 윈스턴은 텔레스크린의 시야에서 벗어나기 위해 거실에 있는 알코브에 앉아 있습니다. 또한, 윈스턴이 소유한 특별한 책은 과거에 제작된 것으로, 그것을 소유하는 것은 위험한 일이라는 것을 암시하고 있습니다.생각경찰과 같은 국가의 감시체계에 반항하는 윈스턴의 행동은 이 소설의 중요한 부분을 이루고 있습니다."

In [8]:
chain.run("Victory Mansions에 대한 설명을 주세요.")

'Victory Mansions는 조지 오웰의 소설 "1984"에 등장하는 픽셀 홈입니다. 이 곳은 윈스턴 스미스가 살고 있는 곳으로, 가난하고 혼란스럽고 통제되는 사회적 환경을 반영하기 위해 설명됩니다. 건물 내부는 낡고 냄새나는 곳이며, 전기가 절약을 위해 끊겨 있고 벽에는 빅 브러더를 찬양하는 포스터가 붙어 있습니다. 아파트는 엘리베이터가 잘 작동하지 않는 등 여러 불편한 상황이 묘사되어 있습니다.'

## Recap
### File Loader
- 1. text loader를 이용해서 file 들을 load하는 방법을 배움   
- 2. UnstructuredFileLoader에 대해서 살펴봄.
    - 이걸로 다양한 file들을 load가능함! text , doc , pdf , html , excel 등등
- 3. file 분할(split)하는 방법도 배움. 아주 긴text를 다루기 위해서 였음.
- 4. 긴 text를 작은 document로 나누는 작업을 했음.
    - 거대한 단일 document보다는, 작은 여러개의 document를 LLM에게 전달할 때 검색 성능이 올라가는 이유가 있기 떄문임.
    - document를 작게 분할하면, 작업이 쉬워지고 응답도 빨라지고 LLM사용 비용 또한 저렴해짐.
    - 작업을 요악하자면, document를 적재(load)하고 분할(split)했음.

### Embedding
- 1. text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한것. (vector 3차원 점수)
- 2. `OpenAIEmbeddings model`을 사용 했었음.
- 3. `ChacheBackedEmbedding`을 사용하여 만들어진 `Embedding`을 cache(저장)했음. (Embedding은 공짜가 아니기 떄문임. GPT3,4는 아니지만 )
- 4. `CacheBackedEmbeddings Class`를 사용했음, 여기에 `Embeddings model`을 전달하고, 데이터가 될 cahe된 directory(폴더)를 지정해 줬음.
- 5. 다음으로는 Vector store를 호출(call)해줬음(FAISS 사용)
- 6. FAISS에서는 document와 embeddding와 함께 from_documents메소드를 호출함.
    - from_documents메소드는 document별로 embeddding 작업 후 결과를 저장한 vector store를 반환하는데, 이를 이용해 document검색도 하고, 연관성이 높은(ralated) document들을 찾기도 했음.

### Retrieval 🪄
- 1. `RetrievalQA`라는 Chain으로 부터 시작됨. 
- 2. 필요한 입력값은 LLM, chain의 종류의 `chain_type`, 그리고 `retriver`임.
- 3. retriver는 Langchain이 제공하는 class 또는 interface의 일종인데, document를 검색해서 찾아오는(retrieve)기능을 가지고 있음.
- 4. 많은 것들이 retriver가 될 수 있음, vector store를 이용해서 만들 수도 있음.
- 5. `as_retriever`메서드 호출 만으로 vector store를 retriever로 전환 가능함.
- 6. 여러 chain_type이 있었음
    - `Stuff`로 시작함. 검색(search)을 통해 related document들을 얻어서, prompt에 입력한 후 그 document들과 함께 LLM에게 '질문에 답하세요'라는 요청을 보냄.
    - `MapReduce`의 작동도 봤었음. MapReduce는 각 document를 `순회`하면서 개별 답변을 찾아낸 후, 탐색이 끝나면, 일종의 중간(intermediate)답변들을 기반으로 최종 응답을 반환해줌.
    - `MapReRank`의 작동도 살펴봄. 이것도 비슷하게 각 document를 `순회`하면서 작업을 시작하는데, 차이점은, 단순히 document의 답변을 추출하는 대신 각 document에 기반해서 질문에 대답하고, `답변에 점수를 매겨줌.` 
    만약 10개의 document가 있다면, 개별 document를 이용한 각각의 답변을 생성하고, `답변별로 점수를 매긴다음`, `가장 높은 점수의 답변을 반환 해주는 방식.`
    - `Refine`도 확인해봄, refine이 하는일은 `document들을 읽으면서` 답변을 생성하고, `읽은 document들을 기반으로 처음의 답변을 업데이트하고,` 정제해줌(refine: 정제 , 다듬기) 점점 좋아지는 답변.
